In [1]:
import os
import tqdm
import copy
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from alignment_helper_fns import *
from gop_helper_fns import *
from mfa_evaluation_utils import *
from g2p_en import G2p
g2p = G2p()

/home/prad/anaconda3/envs/a2/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/prad/anaconda3/envs/a2/lib/python3.7/site-packages/transformers/configuration_utils.py:359: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
/home/prad/anaconda3/envs/a2/lib/python3.7/site-packages/Bio/pairwise2.py:283: BiopythonDeprecationWarning: Bio.pairwise2 has been d

In [2]:
satdf = pd.read_csv('./results_sat/results.csv').set_index('Unnamed: 0')
xsatdf = pd.read_csv('./results_sat_xvector/results_xvector_proj.csv')
xsatdf = xsatdf.rename(columns={'Unnamed: 0': 'Speaker'})

ALL_SPEAKERS = list(satdf.index)

results_dir_frame = './results_frame_10epochs'
results_dir_ivec = './results_sat'
# results_dir_xvec = './results_sat_xvector'
results_dir_xvec = './phone_matched_xvec_proj_textgrids'

In [3]:
EXCLUDE_FILES = ['0505_M_EKs4T10', '0411_M_LMwT32']

allmanual_tgs = [pth for pth in get_all_textgrids_in_directory('/home/prad/datasets/ChildSpeechDataset/manually-aligned-text-grids/') if '.TextGrid' in pth]
allmanual_tgs = [tg for tg in allmanual_tgs if all([_excludefile not in tg for _excludefile in EXCLUDE_FILES])]

ivector_tgs =[pth for pth in get_all_textgrids_in_directory('./results_sat') if '.TextGrid' in pth]
ivector_tgs = [tg for tg in ivector_tgs if all([_excludefile not in tg for _excludefile in EXCLUDE_FILES])]


xvector_tgs =[pth for pth in get_all_textgrids_in_directory('./phone_matched_xvec_proj_textgrids') if '.TextGrid' in pth]
# xvector_tgs =[pth for pth in get_all_textgrids_in_directory('./results_xvector_reevaluated') if '.TextGrid' in pth]
xvector_tgs = [tg for tg in xvector_tgs if all([_excludefile not in tg for _excludefile in EXCLUDE_FILES])]

mfa_tgs = [pth for pth in get_all_textgrids_in_directory('./results_mfa_adapted') if '.TextGrid' in pth]
mfa_tgs = [tg for tg in mfa_tgs if all([_excludefile not in tg for _excludefile in EXCLUDE_FILES])]

print('Num Mismatched:\t', sum([f1.split('/')[-1].split('.')[0]!=f2.split('/')[-1].split('.')[0] for f1, f2 in zip(allmanual_tgs, xvector_tgs)]))

print(len(allmanual_tgs))
print(len(ivector_tgs))
print(len(xvector_tgs))
print(len(mfa_tgs))

Extracting all textgrids in directory:	 /home/prad/datasets/ChildSpeechDataset/manually-aligned-text-grids/


43it [00:00, 6417.19it/s]


Extracting all textgrids in directory:	 ./results_sat


43it [00:00, 6064.80it/s]


Extracting all textgrids in directory:	 ./phone_matched_xvec_proj_textgrids


43it [00:00, 6296.87it/s]


Extracting all textgrids in directory:	 ./results_mfa_adapted


0it [00:00, ?it/s]

Num Mismatched:	 0
3762
3762
3762
0


In [36]:
method_to_dir = {
                 # 'mfa_adapt_from_trained':'./results_mfa_adapted_from_trained_sat3', 
                 # 'mfa_adapt_us_arpa':'./results_mfa_adapted_english_us_arpa',
                 # 'mfa_adapt':'./results_mfa_adapted', 
                 # 'mfa_train':'./results_mfa_train',
                 # 'mfa_trained_old': './results_mfa_trained',
                 # 'mfa_base': './results_mfa_base',
                 # 'ivector': './results_sat', 
                 # 'xvector': './phone_matched_xvec_proj_textgrids', 
                 'xvectorv2': './results_sat_xvector',
                 # 'frame': './results_frame_10epochs', 
                 'gt': '/home/prad/datasets/ChildSpeechDataset/manually-aligned-text-grids/'}

methodnames = list(method_to_dir.keys())

In [37]:
def calc_alignment_accuracy_between_textgrids(manual_textgridpath: str, aligner_textgridpath: str, manual_phonekey: str,
                                              aligner_phonekey: str, collapse_shortphones: bool, remove_numbers=True, ignore_extras=True,
                                              ignore_silence=False, ):
    #TODO: return the label along with whether it was correct so that you can figure out which phonemes were wrong

    manualdf = textgridpath_to_phonedf(manual_textgridpath, phone_key=manual_phonekey, remove_numbers=True)
    alignerdf = textgridpath_to_phonedf(aligner_textgridpath, phone_key=aligner_phonekey, remove_numbers=True,
                                        replace_silence=ignore_silence)

    transcript = get_transcript_from_tgfile(aligner_textgridpath)
    if collapse_shortphones:
        alignerdf = process_silences(alignerdf, transcript)

    phn_midpoints_dict = extract_phn_midpoint_dict_from_df(manualdf)
    correct_indicator = calc_accuracy(predphn_df=alignerdf, annotated_midpoints_dict=phn_midpoints_dict,
                                      ignore_extras=ignore_extras)
    #     if any([c==-1 for c in correct_indicator]):
    if type(correct_indicator) == int:
        print('---------------------------------------')
        print('**********Manual************\n', manualdf)
        print('**********Aligned***********\n', alignerdf)
    return correct_indicator

In [38]:

def contains_same_phones(phone_list1, phone_list2):
    '''
        simply checks that two phoneme lists are the same
        for example that a ground truth textgrid and an estimated textgrid have the same list of phonemes
    '''
    try:
        # return np.logical_and.reduce(np.unique(phone_list1)==np.unique(phone_list2))    
        return np.all(np.array(phone_list1)==np.array(phone_list2))
    except:
        return False
    

def calc_acc_between_tg_lists(manual_tg_list, estimated_tg_list, collapse_shortphones: bool,  manual_phonekey='ha phones', aligner_phonekey='phones', 
                             ignore_numbers=True, ignore_extras=True, ignore_silence=False, verbose=True):
    
    
    correct_indicator = []
    correct_indicator_matched = []
    matched_indicator = []
    for ii, (manual_textgridpath, estimated_textgridpath) in tqdm.tqdm(enumerate(zip(manual_tg_list, estimated_tg_list))):
        # try:
        _correct_indicator = calc_alignment_accuracy_between_textgrids(manual_textgridpath = manual_textgridpath,
                                                                      aligner_textgridpath = estimated_textgridpath,
                                                                      manual_phonekey=manual_phonekey, 
                                                                      aligner_phonekey=aligner_phonekey, 
                                                                      ignore_extras=ignore_extras, 
                                                                      ignore_silence=ignore_silence, 
                                                                      collapse_shortphones=False)

        correct_indicator.append(_correct_indicator)


        manualdf = textgridpath_to_phonedf(manual_textgridpath, phone_key=manual_phonekey, remove_numbers=True)
        alignerdf = textgridpath_to_phonedf(estimated_textgridpath, phone_key=aligner_phonekey, remove_numbers=True,
                                    replace_silence=True)
        transcript = get_transcript_from_tgfile(estimated_textgridpath)

        manual_phones = np.array(remove_sil_from_phonelist(manualdf.phone.values))
        aligner_phones = np.array(remove_sil_from_phonelist(alignerdf.phone.values))
        ismatch = contains_same_phones(manual_phones, aligner_phones)
        matched_indicator.append(ismatch)

        if ismatch:
            correct_indicator_matched.append(_correct_indicator)
            
        # if ii>25:
            # break
        # print(manualdf)
        # print(alignerdf)
        # print(correct_indicator)
        # except:
            # print('error')
            # if not os.path.exists(estimated_textgridpath):
                # print('Textgrid file', estimated_textgridpath, 'not found, skipping this file')
    # print(correct_indicator)
    
    # return 0
    correct_indicator = np.concatenate(correct_indicator)
    acc = np.mean(correct_indicator)
    numcorrect = np.sum(correct_indicator)
    numpredicted = len(correct_indicator)

    correct_indicator_matched = np.concatenate(correct_indicator_matched)    
    acc_matched = np.mean(correct_indicator_matched)
    numcorrect_matched = np.sum(correct_indicator_matched)
    numpredicted_matched = len(correct_indicator_matched)
    
    if verbose:
        print('============ Total Accuracy ============')
        print('Accuracy:\t', np.mean(correct_indicator))
        print('Num Correct:\t', numcorrect)
        print('Num Predicted Phones:\t', numpredicted)
        print('============ Matched Accuracy ============')
        print('Matched Accuracy:\t', np.mean(correct_indicator_matched))
        print('Num Correct Matched:\t', numcorrect_matched)
        print('Num Predicted Phones Matched:\t', numpredicted_matched)
    
    return acc, acc_matched, numcorrect, numcorrect_matched, numpredicted, numpredicted_matched, matched_indicator
    
    
def calculate_accuracy_for_methods(speakers, methodname, method_to_dir=method_to_dir):
    
    correct_indicator_matched = []
    correct_indicator = []
    
    all_gt_tgs = []
    all_method_tgs = []
    
    for speakerid in speakers:
        tgs_manualpath = os.path.join(method_to_dir['gt'], speakerid)
        tgs_methodpath = os.path.join(method_to_dir[methodname], speakerid)
        tgs_manual = get_all_textgrids_in_directory(tgs_manualpath, verbose=False)
        tgs_method = get_all_textgrids_in_directory(tgs_methodpath, verbose=False)
    
        all_gt_tgs.extend(tgs_manual)
        all_method_tgs.extend(all_method_tgs)
        
    acc, acc_matched, numcorrect, numcorrect_matched, numpredicted, numpredicted_matched = \
        calc_acc_between_tg_lists(tgs_manual, tgs_method, collapse_shortphones=True,  manual_phonekey='ha phones', aligner_phonekey='phones')

In [39]:
speakers = ALL_SPEAKERS
# methodname='mfa_train'
# methodname='mfa_trained_old'
# methodname = 'xvector'
collapse_shortphones=False
acc_results_dct = {}

for methodname in method_to_dir.keys():
    if methodname=='gt':
        continue
    print('-------------------------------------------------------')
    print('Methodname:', methodname)
    correct_indicator_matched = []
    correct_indicator = []

    all_gt_tgs = []
    all_method_tgs = []

    for speakerid in speakers:
        tgs_manualpath = os.path.join(method_to_dir['gt'], speakerid)
        tgs_methodpath = os.path.join(method_to_dir[methodname], speakerid)
        tgs_manual = get_all_textgrids_in_directory(tgs_manualpath, verbose=False)
        tgs_method = get_all_textgrids_in_directory(tgs_methodpath, verbose=False)

        all_gt_tgs.extend(tgs_manual)
        all_method_tgs.extend(tgs_method)

    acc, acc_matched, numcorrect, numcorrect_matched, numpredicted, numpredicted_matched, matched_indicator = \
        calc_acc_between_tg_lists(all_gt_tgs, all_method_tgs, collapse_shortphones=collapse_shortphones,  manual_phonekey='ha phones', aligner_phonekey='phones')
    acc_results_dct[methodname] = {'Acc': acc, 
                                   'NumCorrect':numcorrect, 
                                   'NumPredicted':numpredicted,
                                   'AccMatched': acc_matched,
                                   'NumCorrectMatched':numcorrect_matched, 
                                   'NumPredictedMatched':numpredicted_matched}

-------------------------------------------------------
Methodname: xvectorv2


3763it [00:19, 192.49it/s]

============ Total Accuracy ============
Accuracy:	 0.5412100684067211
Num Correct:	 8149.0
Num Predicted Phones:	 15057
============ Matched Accuracy ============
Matched Accuracy:	 0.9449916372421483
Num Correct Matched:	 5085
Num Predicted Phones Matched:	 5381


In [26]:
acc_results_df = pd.DataFrame.from_dict(acc_results_dct).T
acc_results_df.to_csv('./interspeech_results/acc_results.csv')

In [27]:
acc_results_df

,Acc,NumCorrect,NumPredicted,AccMatched,NumCorrectMatched,NumPredictedMatched
mfa_adapt_from_trained,0.606486,561.0,925.0,0.627483,379.0,604.0
mfa_adapt_us_arpa,0.721616,661.0,916.0,0.719611,444.0,617.0
mfa_train,0.606486,561.0,925.0,0.627483,379.0,604.0
mfa_trained_old,0.878289,801.0,912.0,0.906937,536.0,591.0
mfa_base,0.710383,650.0,915.0,0.706645,436.0,617.0
ivector,0.918542,857.0,933.0,0.947368,522.0,551.0
xvector,0.928799,874.0,941.0,0.935867,788.0,842.0
frame,0.901499,842.0,934.0,0.928437,493.0,531.0


In [28]:
remove_indices = ['mfa_train', 'mfa_adapt_from_trained', 'mfa_adapt_us_arpa'] 
acc_results_df.drop(remove_indices)

,Acc,NumCorrect,NumPredicted,AccMatched,NumCorrectMatched,NumPredictedMatched
mfa_trained_old,0.878289,801.0,912.0,0.906937,536.0,591.0
mfa_base,0.710383,650.0,915.0,0.706645,436.0,617.0
ivector,0.918542,857.0,933.0,0.947368,522.0,551.0
xvector,0.928799,874.0,941.0,0.935867,788.0,842.0
frame,0.901499,842.0,934.0,0.928437,493.0,531.0
